# The Mighty Thymio: Actuators

In [2]:
import rospy

In [3]:
import rosnode
import rostopic
import rosmsg

In [23]:
import std_msgs.msg
import thymio_msgs.msg
import geometry_msgs.msg

In [5]:
rospy.init_node('notebook_actuators')

In [6]:
def print_msg_definition(msg_type):
    print(rosmsg.get_msg_text(msg_type))

# [Raw] Aseba Messages

Aseba message are of two types: 
    - asebaros_msgs/AsebaEvent
    - asebaros_msgs/AsebaAnonymousEvent

In [9]:
import asebaros_msgs.msg

In [14]:
print_msg_definition('asebaros_msgs/AsebaEvent')

time stamp
uint16 source
int16[] data



In [16]:
pub = rospy.Publisher('/aseba/events/set_speed', asebaros_msgs.msg.AsebaEvent, queue_size=1)

In [ ]:
pub.publish(asebaros_msgs.msg.AsebaEvent(data=[100,100], source=0, stamp=rospy.Time.now()))

In [17]:
pub.publish(asebaros_msgs.msg.AsebaEvent(data=[0,0], source=0, stamp=rospy.Time.now()))

# Actuators

## Motors

To send a velocity command to the robot, send a `geometry_msgs/Twist` message on `cmd_vel`

In [18]:
print(rosmsg.get_msg_text("geometry_msgs/Twist"))

geometry_msgs/Vector3 linear
  float64 x
  float64 y
  float64 z
geometry_msgs/Vector3 angular
  float64 x
  float64 y
  float64 z



In [20]:
pub = rospy.Publisher('cmd_vel', geometry_msgs.msg.Twist, queue_size=1)
msg = geometry_msgs.msg.Twist()

In [22]:
msg.linear.x = 0.02
pub.publish(msg)
rospy.sleep(1)
msg.linear.x = 0
pub.publish(msg)

In [ ]:
## TODO /comm/transmit

## LEDs

The robot has several leds:
    - 8 yellow [state] leds around the buttons
    - 4 red leds near on of each button
    - 8 red leds near the proximity sensors (6 front and 2 rear)
    - 2 red leds near the ground sensors
    - one violet led near the temperature sensors
    - one blue led near the microphone
    - one red led near the IR receiver
    
that can be directly set to a value between 0 (off) and 1 (full light) with a `thymio_msgs/Led` on `/led`.

In [24]:
print_msg_definition('thymio_msgs/Led')

uint8 CIRCLE=0
uint8 PROXIMITY=1
uint8 BUTTONS=2
uint8 GROUND=3
uint8 TEMPERATURE=4
uint8 MICROPHONE=5
uint8 REMOTE=6
float32[] values
uint8 id



In [25]:
pub = rospy.Publisher('led', thymio_msgs.msg.Led, queue_size=1)

In [45]:
led_groups = ['CIRCLE', 'PROXIMITY', 'BUTTONS','GROUND','TEMPERATURE', 'MICROPHONE', 'REMOTE']

for led_group in led_groups:
    print('Switch leds in {led_group} on'.format(**locals()))
    id_ = getattr(thymio_msgs.msg.Led, led_group)
    pub.publish(id=id_, values=[1]*8)
    rospy.sleep(2)
    print('Switch leds in {led_group} off'.format(**locals()))
    pub.publish(id=id_, values=[0]*8)
    rospy.sleep(2)

Switch leds in CIRCLE on
Switch leds in CIRCLE off
Switch leds in PROXIMITY on
Switch leds in PROXIMITY off
Switch leds in BUTTONS on
Switch leds in BUTTONS off
Switch leds in GROUND on
Switch leds in GROUND off
Switch leds in TEMPERATURE on
Switch leds in TEMPERATURE off
Switch leds in MICROPHONE on
Switch leds in MICROPHONE off
Switch leds in REMOTE on
Switch leds in REMOTE off


The robots has also three larger leds that change the RGB colors of its (translucent) plastic body
    - /led/body/bottom_left
    - /led/body/bottom_right
    - /led/body/top
with a `std_msgs/ColorRGBA` messag (the A component of the color is ignored by the robot).

In [56]:
print_msg_definition('std_msgs/ColorRGBA')

float32 r
float32 g
float32 b
float32 a



In [59]:
colors = {'bottom_left': [0.1, 0.9, 0.1],
          'bottom_right': [1, 0.3, 0.8], 
          'top': [0.0, 0.1, 0.4]}

for led, color  in colors.items():
    pub = rospy.Publisher('led/body/{led}'.format(**locals()), std_msgs.msg.ColorRGBA, queue_size=1)
    print('Switch body {led} on'.format(**locals()))
    rgba = list(color) + [1]
    pub.publish(*rgba)

Switch body top on
Switch body bottom_left on
Switch body bottom_right on


### Led animations

The robot is also able to display animation (gestures) by sending a message `thymio_msgs/LedGesture` to `/led/gesture`, which are implemented as waves with period (in seconds) and length (in number of leds), and where
    - mask[i] = True => the led will be set to 0,
    - mirror = True => The wave is (spatially) mirrored, like a standing wave.
    - leds is one of the led groups
    - wave is either RECT (a rectified wave) or HARMONIC (a sinosoundal wave)
    - gesture is either OFF (set off all leds) or WAVE



In [60]:
print_msg_definition('thymio_msgs/LedGesture')

uint8 OFF=0
uint8 WAVE=1
uint8 RECT=0
uint8 HARMONIC=1
uint8 CIRCLE=0
uint8 PROXIMITY=1
uint8 BUTTONS=2
uint8 gesture
uint8 wave
uint8 leds
float32 period
uint16 length
uint8 mirror
bool[] mask



In [61]:
pub = rospy.Publisher('led/gesture', thymio_msgs.msg.LedGesture, queue_size=1)

#### Type of wave

In [87]:
pub.publish(mask=[1]*8, mirror=False, length=8, period=1, leds=thymio_msgs.msg.LedGesture.CIRCLE, 
            gesture=thymio_msgs.msg.LedGesture.WAVE, wave=thymio_msgs.msg.LedGesture.RECT)
rospy.sleep(5)
pub.publish(mask=[1]*8, mirror=False, length=6, period=1, leds=thymio_msgs.msg.LedGesture.CIRCLE, 
            gesture=thymio_msgs.msg.LedGesture.WAVE, wave=thymio_msgs.msg.LedGesture.HARMONIC)

#### Leds

In [88]:
pub.publish(mask=[1]*8, mirror=False, length=6, period=1, leds=thymio_msgs.msg.LedGesture.CIRCLE, 
            gesture=thymio_msgs.msg.LedGesture.OFF, wave=thymio_msgs.msg.LedGesture.RECT)

pub.publish(mask=[1]*8, mirror=False, length=6, period=1, leds=thymio_msgs.msg.LedGesture.PROXIMITY, 
            gesture=thymio_msgs.msg.LedGesture.WAVE, wave=thymio_msgs.msg.LedGesture.HARMONIC)

### Length

In [107]:
pub.publish(mask=[1]*8, mirror=False, length=8, period=1, leds=thymio_msgs.msg.LedGesture.CIRCLE, 
            gesture=thymio_msgs.msg.LedGesture.WAVE, wave=thymio_msgs.msg.LedGesture.HARMONIC)

rospy.sleep(3)

pub.publish(mask=[1]*8, mirror=False, length=4, period=1, leds=thymio_msgs.msg.LedGesture.CIRCLE, 
            gesture=thymio_msgs.msg.LedGesture.WAVE, wave=thymio_msgs.msg.LedGesture.HARMONIC)

rospy.sleep(3)

pub.publish(mask=[1]*8, mirror=False, length=0, period=1, leds=thymio_msgs.msg.LedGesture.CIRCLE, 
            gesture=thymio_msgs.msg.LedGesture.WAVE, wave=thymio_msgs.msg.LedGesture.HARMONIC)

### Period

In [108]:
pub.publish(mask=[1]*8, mirror=False, length=5, period=1, leds=thymio_msgs.msg.LedGesture.CIRCLE, 
            gesture=thymio_msgs.msg.LedGesture.WAVE, wave=thymio_msgs.msg.LedGesture.HARMONIC)

rospy.sleep(3)

pub.publish(mask=[1]*8, mirror=False, length=5, period=3, leds=thymio_msgs.msg.LedGesture.CIRCLE, 
            gesture=thymio_msgs.msg.LedGesture.WAVE, wave=thymio_msgs.msg.LedGesture.HARMONIC)

rospy.sleep(3)

pub.publish(mask=[1]*8, mirror=False, length=5, period=0.3, leds=thymio_msgs.msg.LedGesture.CIRCLE, 
            gesture=thymio_msgs.msg.LedGesture.WAVE, wave=thymio_msgs.msg.LedGesture.HARMONIC)

### Mirror

In [113]:
# pub.publish(mask=[1]*8, mirror=True, length=8, period=2, leds=thymio_msgs.msg.LedGesture.CIRCLE, 
#             gesture=thymio_msgs.msg.LedGesture.WAVE, wave=thymio_msgs.msg.LedGesture.HARMONIC)

### Predefined gestures

In [ ]:
There are some predefined gestures, like
    - /led/gesture/alive
    - /led/gesture/blink
    - /led/gesture/circle
    - /led/gesture/kit
    - /led/gesture/off

In [115]:
pub = rospy.Publisher('/led/gesture/alive', std_msgs.msg.Empty, queue_size=1)
pub.publish()

In [118]:
pub = rospy.Publisher('/led/gesture/blink', std_msgs.msg.Float32, queue_size=1)
period = 0.5
pub.publish(period)

In [121]:
pub = rospy.Publisher('/led/gesture/circle', std_msgs.msg.Float32, queue_size=1)
period = 2
pub.publish(period)

In [127]:
pub = rospy.Publisher('/led/gesture/kit', std_msgs.msg.Float32, queue_size=1)
period = 1.8
pub.publish(period)

In [129]:
pub = rospy.Publisher('/led/gesture/off', std_msgs.msg.Empty, queue_size=1)
pub.publish()

## Sounds

To play a sound, you can either send a `thymio_msgs/Sound` message to `sound/play`

In [131]:
print_msg_definition('thymio_msgs/Sound')

duration duration
float32 frequency



In [155]:
pub = rospy.Publisher('/sound/play', thymio_msgs.msg.Sound, queue_size=1)
for _ in range(3):
    pub.publish(duration=rospy.Duration(0.5), frequency=164.81)
    rospy.sleep(0.6)
pub.publish(duration=rospy.Duration(1.7), frequency=130.81)
rospy.sleep(1.8)
for _ in range(3):
    pub.publish(duration=rospy.Duration(0.5), frequency=146.83)
    rospy.sleep(0.6)
pub.publish(duration=rospy.Duration(1.7), frequency=123.47)


or you can play a prerecorded system sound on `sound/play/system`

In [156]:
print_msg_definition('thymio_msgs/SystemSound')

int8 stop=-1
int8 startup=0
int8 shutdown=1
int8 arrow_button=2
int8 central_button=3
int8 free_fall=4
int8 collision=5
int8 target_ok=6
int8 target_detect=7
int8 sound



In [157]:
pub = rospy.Publisher('/sound/play/system', thymio_msgs.msg.SystemSound, queue_size=1)

In [158]:
pub.publish(sound=thymio_msgs.msg.SystemSound.central_button)